<a href="https://colab.research.google.com/github/Djensonsan/Information_Retrieval_Assignment_2/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Imports

In [1]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs dask
!pip install -q tqdm

In [2]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import dask.dataframe as dd

import gensim
from tqdm import tqdm
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [ ]:
# Pycharm:
# data = pd.read_csv('data/news_dataset.csv')

In [5]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/news_dataset.csv')

### Exploration

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141585 entries, 0 to 141584
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           141585 non-null  int64 
 1   title        141583 non-null  object
 2   publication  141564 non-null  object
 3   author       125725 non-null  object
 4   date         138869 non-null  object
 5   year         138911 non-null  object
 6   month        138911 non-null  object
 7   url          84541 non-null   object
 8   content      141543 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.7+ MB


In [7]:
data.head(n=3)

,id,title,publication,author,date,year,month,url,content
0,1,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016,12,NaN,WASHINGTON — Congressional Republicans have...
1,2,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017,6,NaN,"After the bullet shells get counted, the blood..."
2,3,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017,1,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."


### Keep document content

In [8]:
data_content = data['content']

In [9]:
type(data_content)

pandas.core.series.Series

In [10]:
data_content.head(n=3)

0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
Name: content, dtype: object

### Tokenization, Stemming and Lemmatization

In [11]:
tqdm.pandas()
# Note Jens: Might want to use Dask to speed up. 
# When using Dask can't use tqdm as far as I know.

In [12]:
# Tokenization
data_content_tokenized = data_content[0:100].progress_apply(lambda x: nltk.word_tokenize(x))

100%|██████████| 100/100 [00:01<00:00, 89.93it/s]


In [13]:
# Remove words smaller than 3 characters
data_content_tokenized = data_content_tokenized.progress_apply(lambda x: [y for y in x if len(y)>2])

100%|██████████| 100/100 [00:00<00:00, 4460.98it/s]


In [14]:
# Stemming and Lemmatization 
stemmer = SnowballStemmer("english")
data_content_stemmed = data_content_tokenized.progress_apply(lambda x: [stemmer.stem(WordNetLemmatizer().lemmatize(y)) for y in x])

100%|██████████| 100/100 [00:03<00:00, 28.49it/s]


In [15]:
# Remove Stopswords
stop_words = set(stopwords.words('english')) 
data_content_clean = data_content_stemmed.progress_apply(lambda x: [y for y in x if not y in stop_words])

100%|██████████| 100/100 [00:00<00:00, 6519.68it/s]


In [16]:
data_content_clean.head()

0    [washington, congression, republican, new, fea...
1    [bullet, shell, get, count, blood, dri, votiv,...
2    [walt, disney, bambi, open, 1942, critic, prai...
3    [death, may, great, equal, necessarili, evenha...
4    [seoul, south, korea, north, korea, leader, ki...
Name: content, dtype: object

## LDA Library

## Topic Generation

## Experimentation

## Evaluation

## Top Documents